In [3]:
import pandas as pd
import pyodbc
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [4]:
# Connection string to connect to SQL Server named instance
conn_str = 'Driver=SQL Server;Server=.;Database=RevoScaleDb;Trusted_Connection=True;'
query ='SELECT Facidity, Vacidity, Citric, Sugar, Chlorides, \
               Fsulfur, Tsulfur, Density, pH,Sulphates, Alcohol,\
               CASE WHEN Color =\'red\' THEN 1 ELSE 0 END ColNum,\
               Quality\
        FROM dbo.WineTrain\
        ORDER BY Id';
conn = pyodbc.connect(conn_str)
#Get traing dataset
wines = pd.read_sql(query,conn)

In [5]:
wines_train, wines_test = train_test_split(wines)

In [16]:
wines_test.head()

,Facidity,Vacidity,Citric,Sugar,Chlorides,Fsulfur,Tsulfur,Density,pH,Sulphates,Alcohol,ColNum,Quality
4217,10.0,0.560,0.24,2.2,0.079,19.0,58.0,0.99910,3.18,0.56,10.1,1,6
3293,6.8,0.170,0.35,1.8,0.040,29.0,84.0,0.98961,2.91,0.57,12.0,0,7
4205,6.9,0.765,0.18,2.4,0.243,5.5,48.0,0.99612,3.40,0.60,10.3,1,6
1612,7.8,0.260,0.49,3.2,0.027,28.0,87.0,0.99190,3.03,0.32,11.3,0,7
3281,6.5,0.320,0.30,2.3,0.051,20.0,127.0,0.98964,3.13,0.52,12.8,0,6


In [6]:
from revoscalepy import rx_dtree
features = wines.columns.drop(["Quality"])
model = rx_dtree("Quality ~" + "+".join(features), data=wines_train, method="regression")


oneFilePerTree is not a logical and is being reset to FALSE

allowDiskWrite is not a logical and is being reset to true

useWriteDataSource is not an integer and is being reset to 0

autoCleanup is not a logical and is being reset to TRUE
Rows Read: 3411, Total Rows Processed: 3411, Total Chunk Time: 0.018 seconds 
Rows Read: 3411, Total Rows Processed: 3411, Total Chunk Time: 0.014 seconds 
Rows Read: 3411, Total Rows Processed: 3411, Total Chunk Time: 0.018 secondsoneFilePerTree is not a logical and is being reset to FALSE
 
Rows Read: 2, Total Rows Processed: 2, Total Chunk Time: 0.003 seconds 
Rows Read: 3411, Total Rows Processed: 3411, Total Chunk Time: 0.017 seconds 
Rows Read: 4, Total Rows Processed: 4, Total Chunk Time: 0.005 seconds 
Rows Read: 3411, Total Rows Processed: 3411, Total Chunk Time: 0.022 seconds 
Rows Read: 8, Total Rows Processed: 8, Total Chunk Time: 0.012 seconds 
Rows Read: 3411, Total Rows Processed: 3411, Total Chunk Time: 0.022 seconds 
Rows Read: 16, T

In [7]:
from revoscalepy import rx_predict
pred = rx_predict(model, wines_test, extra_vars_to_write=["Quality"])
print(pred.head())

Rows Read: 1138, Total Rows Processed: 1138, Total Chunk Time: 0.002 seconds 
   Quality_Pred  Quality
0      6.352941      6.0
1      5.369565      5.0
2      6.181818      5.0
3      4.600000      5.0
4      6.750000      7.0


In [19]:
r2 = r2_score(pred.Quality, pred.Quality_Pred)
print("Accuracy: {}%".format(int(round(r2 * 100))))

Accuracy: 30%


In [8]:
from revoscalepy import rx_serialize_model
rx_trees_model = rx_serialize_model(model, realtime_scoring_only = True);

In [9]:
#Save model
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()
cursor.execute("DELETE FROM dbo.Models WHERE ModelName ='rxDTree' AND ModelLanguage ='RevoScalePy'");
cursor.execute("INSERT INTO dbo.Models(ModelName,ModelLanguage,Model) values (?,?,?)", 
               'rxDTree','RevoScalePy',rx_trees_model)
conn.commit()